# Getting Started with Microsoft Agent Framework

Welcome! This notebook demonstrates how to use Microsoft Agent Framework for building AI agents.

## What is Agent Framework?

Agent Framework is Microsoft's latest framework for building AI agents with these key features:

- **Simpler API**: Clean, intuitive interfaces with minimal boilerplate
- **Provider Agnostic**: Works with any chat provider (Azure OpenAI, OpenAI, local models, etc.)
- **Unified Threading**: Single `AgentThread` abstraction for conversations
- **Built-in Tool Support**: Native support for extending agent capabilities
- **Azure-optimized**: First-class Azure support while remaining provider-neutral
- **Production Ready**: Enterprise-grade features and stability

In [ ]:
# Install required packages (run this cell first)
%pip install agent-framework python-dotenv

## Setting Up Agent Framework Environment

Agent Framework has a straightforward setup. Below we'll demonstrate setting up chat clients for different providers.

### Environment Configuration

Before running the examples, you need to configure your environment:

1. Copy `.env.example` to `.env` in the workspace root
2. Fill in your actual credentials (Azure OpenAI endpoint, deployment name, API key)
3. The notebook will load these automatically using `python-dotenv`

**Note**: Never commit `.env` to version control - it's already in `.gitignore`

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework import ChatAgent

# Load environment variables (if any)
load_dotenv()

# Example 1: Azure OpenAI Setup
from agent_framework.azure import AzureOpenAIChatClient
azure_client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# Example 2: OpenAI Setup
#from agent_framework.openai import OpenAIChatClient
# openai_client = OpenAIChatClient(
#  model_id="gpt-5-mini",  # or any other OpenAI model
#  api_key=os.getenv("OPENAI_API_KEY")


# Example 3: Local Setup with Ollama
# from agent_framework.ollama import OllamaChatClient
# local_client = OllamaChatClient(
#     base_url="http://localhost:11434",
#     model="llama2"  # or any other local model
# )

## Creating Basic Agents with Agent Framework

Agent Framework uses a simple, unified approach for creating agents. Compare this with Semantic Kernel's more complex plugin and skill system:

In [ ]:
# Create a simple agent with Agent Framework
simple_agent = ChatAgent(
    chat_client=azure_client,  # Use any chat client (Azure, OpenAI, local, etc.)
    name="SimpleAgent",
    instructions="You are a helpful AI assistant that answers questions clearly and concisely."
)

# Create a thread for conversation
thread = simple_agent.get_new_thread()

# Run the agent (using top-level await)
response = await simple_agent.run("What's the capital of France?", thread=thread)
print("Agent:", response.text)

# Continue the conversation in the same thread
response = await simple_agent.run("What's its population?", thread=thread)
print("Agent:", response.text)

## Advanced Agent Features

Let's explore some more advanced capabilities of Agent Framework:

In [ ]:
# Example 1: Dynamic Instructions
specialized_agent = ChatAgent(
    chat_client=azure_client,
    name="PythonExpert",
    instructions="You are a Python expert. Provide concise, practical code examples."
)

response = await specialized_agent.run("Show me how to read a CSV file with pandas")
print("Expert Agent:", response.text)

# Example 2: Streaming Responses (for real-time output)
async for chunk in specialized_agent.run_stream("Explain list comprehensions"):
    print(chunk.text, end="", flush=True)
print()  # New line after streaming

## Enabling Multi-Turn Conversations

One of Agent Framework's most powerful features is built-in conversation threading. Unlike stateless API calls, threads maintain context across multiple interactions, enabling natural back-and-forth conversations.

### Why Use Threads?

- **Context Retention**: The agent remembers previous messages in the conversation
- **Natural Flow**: Ask follow-up questions without repeating context
- **Session Management**: Each thread is independent - perfect for different users or topics
- **Automatic History**: Message history is maintained automatically

In [ ]:
# Example 1: Simple multi-turn conversation
print("=== Example 1: Conversation with Context ===\n")

# Create a new thread for this conversation
conversation_thread = simple_agent.get_new_thread()

# First message
response = await simple_agent.run(
    "What country has Paris as its capital?", 
    thread=conversation_thread
)
print("User: What country has Paris as its capital?")
print(f"Agent: {response.text}\n")

# Follow-up question - agent remembers the context
response = await simple_agent.run(
    "What are its neighboring countries?", 
    thread=conversation_thread
)
print("User: What are its neighboring countries?")
print(f"Agent: {response.text}\n")

# Another follow-up - building on previous context
response = await simple_agent.run(
    "Which one is the largest by area?", 
    thread=conversation_thread
)
print("User: Which one is the largest by area?")
print(f"Agent: {response.text}\n")

print(f"✅ Conversation complete! The agent maintained context across all questions.")

### Multiple Independent Threads

You can create multiple threads to manage different conversations or topics simultaneously. Each thread maintains its own context:

In [ ]:
# Example 2: Multiple independent conversations
print("=== Example 2: Parallel Conversations ===\n")

# Thread 1: Discussing Python
thread_python = simple_agent.get_new_thread()
response = await simple_agent.run("Tell me about Python programming", thread=thread_python)
print("Thread 1 (Python topic):")
print(f"User: Tell me about Python programming")
print(f"Agent: {response.text[:100]}...\n")

# Thread 2: Discussing JavaScript (completely separate context)
thread_javascript = simple_agent.get_new_thread()
response = await simple_agent.run("Tell me about JavaScript", thread=thread_javascript)
print("Thread 2 (JavaScript topic):")
print(f"User: Tell me about JavaScript")
print(f"Agent: {response.text[:100]}...\n")

# Continue thread 1 - no JavaScript context leakage
response = await simple_agent.run("What are its key features?", thread=thread_python)
print("Thread 1 continued:")
print(f"User: What are its key features?")
print(f"Agent: {response.text[:100]}...")
print("(Agent talks about Python, not JavaScript!)\n")

# Continue thread 2 - no Python context leakage
response = await simple_agent.run("What frameworks are popular?", thread=thread_javascript)
print("Thread 2 continued:")
print(f"User: What frameworks are popular?")
print(f"Agent: {response.text[:100]}...")
print("(Agent talks about JavaScript frameworks!)\n")

### Understanding Thread State

Threads in Agent Framework maintain conversation context automatically. You don't need to manually manage message history - the framework handles it for you:

In [ ]:
# Example 3: Understanding thread state
print("=== Example 3: Thread State ===\n")

# Threads maintain conversation context internally
# The Agent Framework handles message history automatically
# You don't need to manually access or manage messages

# Each call to agent.run() with a thread:
# 1. Adds your message to the thread's internal history
# 2. Sends the full conversation context to the model
# 3. Receives and stores the assistant's response
# 4. Returns the response to you

print("The thread object maintains internal state that enables:")
print("  ✓ Context retention across multiple turns")
print("  ✓ Natural conversation flow")
print("  ✓ No manual message management needed")
print("\nJust create a thread once and reuse it for the entire conversation!")

### Exercise: Build a Multi-Turn Conversation

**Your Task**: Create a conversation where:
1. Create a new thread
2. Ask the agent about a topic (e.g., "What is machine learning?")
3. Ask a follow-up question that relies on context (e.g., "What are its main types?")
4. Ask another follow-up (e.g., "Give me an example of supervised learning")
5. Observe how the agent maintains context

**Success Criteria**:
- The agent should understand context in follow-up questions without you repeating information
- Follow-up questions like "What are its types?" should be understood in context

In [ ]:
# Exercise: Your code here
# TODO: Create a multi-turn conversation following the instructions above

# Uncomment and complete:
# my_thread = simple_agent.get_new_thread()
# response = await simple_agent.run("Your first question", thread=my_thread)
# print(f"Agent: {response.text}")
# ... continue with follow-up questions ...

pass

<details>
<summary>💡 Click to see solution</summary>

```python
# Solution: Multi-turn conversation example
my_thread = simple_agent.get_new_thread()

# First question
response = await simple_agent.run("What is machine learning?", thread=my_thread)
print(f"Q1: What is machine learning?")
print(f"A1: {response.text}\n")

# Follow-up relying on context (notice we don't say "machine learning" again)
response = await simple_agent.run("What are its main types?", thread=my_thread)
print(f"Q2: What are its main types?")
print(f"A2: {response.text}\n")

# Another follow-up (still relying on context)
response = await simple_agent.run("Give me an example of supervised learning", thread=my_thread)
print(f"Q3: Give me an example of supervised learning")
print(f"A3: {response.text}\n")

print(f"✅ Conversation complete! The agent understood all context.")
```
</details>

## Summary

Microsoft Agent Framework provides:

1. **Simplicity**: 
   - Clean API with minimal setup
   - No complex configuration needed
   - Direct agent creation and usage

2. **Conversation Management**:
   - Built-in thread support
   - Stateful conversations out of the box
   - Easy multi-turn interactions

3. **Provider Flexibility**:
   - Switch providers by changing chat client
   - Same API works everywhere
   - No vendor lock-in

4. **Modern Design**:
   - Async-first architecture
   - Supports latest AI capabilities
   - Streaming support
   - Tool integration ready

## Next Steps

- Explore tool/function calling in the next notebook
- Learn about multi-agent orchestration
- Build production-ready applications with Agent Framework

# Getting Started with Microsoft Agent Framework

Welcome! This notebook demonstrates how to use Microsoft Agent Framework for building AI agents.

## What is Agent Framework?

Agent Framework is Microsoft's latest framework for building AI agents with these key features:

- **Simpler API**: Clean, intuitive interfaces with minimal boilerplate
- **Provider Agnostic**: Works with any chat provider (Azure OpenAI, OpenAI, local models, etc.)
- **Unified Threading**: Single `AgentThread` abstraction for conversations
- **Built-in Tool Support**: Native support for extending agent capabilities
- **Azure-optimized**: First-class Azure support while remaining provider-neutral
- **Production Ready**: Enterprise-grade features and stability

In [ ]:
# Install required packages (run this cell first)
%pip install agent-framework python-dotenv

## Setting Up Agent Framework Environment

Agent Framework has a straightforward setup. Below we'll demonstrate setting up chat clients for different providers.

### Environment Configuration

Before running the examples, you need to configure your environment:

1. Copy `.env.example` to `.env` in the workspace root
2. Fill in your actual credentials (Azure OpenAI endpoint, deployment name, API key)
3. The notebook will load these automatically using `python-dotenv`

**Note**: Never commit `.env` to version control - it's already in `.gitignore`

In [ ]:
import os
from dotenv import load_dotenv
from agent_framework import ChatAgent

# Load environment variables (if any)
load_dotenv()

# Example 1: Azure OpenAI Setup
from agent_framework.azure import AzureOpenAIChatClient
azure_client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

# Example 2: OpenAI Setup
#from agent_framework.openai import OpenAIChatClient
# openai_client = OpenAIChatClient(
#  model_id="gpt-5-mini",  # or any other OpenAI model
#  api_key=os.getenv("OPENAI_API_KEY")


# Example 3: Local Setup with Ollama
# from agent_framework.ollama import OllamaChatClient
# local_client = OllamaChatClient(
#     base_url="http://localhost:11434",
#     model="llama2"  # or any other local model
# )

## Creating Basic Agents with Agent Framework

Agent Framework uses a simple, unified approach for creating agents. Compare this with Semantic Kernel's more complex plugin and skill system:

In [ ]:
# Create a simple agent with Agent Framework
simple_agent = ChatAgent(
    chat_client=azure_client,  # Use any chat client (Azure, OpenAI, local, etc.)
    name="SimpleAgent",
    instructions="You are a helpful AI assistant that answers questions clearly and concisely."
)

# Create a thread for conversation
thread = simple_agent.get_new_thread()

# Run the agent (using top-level await)
response = await simple_agent.run("What's the capital of France?", thread=thread)
print("Agent:", response.text)

# Continue the conversation in the same thread
response = await simple_agent.run("What's its population?", thread=thread)
print("Agent:", response.text)

## Advanced Agent Features

Let's explore some more advanced capabilities of Agent Framework:

In [ ]:
# Example 1: Dynamic Instructions
specialized_agent = ChatAgent(
    chat_client=azure_client,
    name="PythonExpert",
    instructions="You are a Python expert. Provide concise, practical code examples."
)

response = await specialized_agent.run("Show me how to read a CSV file with pandas")
print("Expert Agent:", response.text)

# Example 2: Streaming Responses (for real-time output)
async for chunk in specialized_agent.run_stream("Explain list comprehensions"):
    print(chunk.text, end="", flush=True)
print()  # New line after streaming

## Enabling Multi-Turn Conversations

One of Agent Framework's most powerful features is built-in conversation threading. Unlike stateless API calls, threads maintain context across multiple interactions, enabling natural back-and-forth conversations.

### Why Use Threads?

- **Context Retention**: The agent remembers previous messages in the conversation
- **Natural Flow**: Ask follow-up questions without repeating context
- **Session Management**: Each thread is independent - perfect for different users or topics
- **Automatic History**: Message history is maintained automatically

In [ ]:
# Example 1: Simple multi-turn conversation
print("=== Example 1: Conversation with Context ===\n")

# Create a new thread for this conversation
conversation_thread = simple_agent.get_new_thread()

# First message
response = await simple_agent.run(
    "What country has Paris as its capital?", 
    thread=conversation_thread
)
print("User: What country has Paris as its capital?")
print(f"Agent: {response.text}\n")

# Follow-up question - agent remembers the context
response = await simple_agent.run(
    "What are its neighboring countries?", 
    thread=conversation_thread
)
print("User: What are its neighboring countries?")
print(f"Agent: {response.text}\n")

# Another follow-up - building on previous context
response = await simple_agent.run(
    "Which one is the largest by area?", 
    thread=conversation_thread
)
print("User: Which one is the largest by area?")
print(f"Agent: {response.text}\n")

print(f"✅ Conversation complete! The agent maintained context across all questions.")

### Multiple Independent Threads

You can create multiple threads to manage different conversations or topics simultaneously. Each thread maintains its own context:

In [ ]:
# Example 2: Multiple independent conversations
print("=== Example 2: Parallel Conversations ===\n")

# Thread 1: Discussing Python
thread_python = simple_agent.get_new_thread()
response = await simple_agent.run("Tell me about Python programming", thread=thread_python)
print("Thread 1 (Python topic):")
print(f"User: Tell me about Python programming")
print(f"Agent: {response.text[:100]}...\n")

# Thread 2: Discussing JavaScript (completely separate context)
thread_javascript = simple_agent.get_new_thread()
response = await simple_agent.run("Tell me about JavaScript", thread=thread_javascript)
print("Thread 2 (JavaScript topic):")
print(f"User: Tell me about JavaScript")
print(f"Agent: {response.text[:100]}...\n")

# Continue thread 1 - no JavaScript context leakage
response = await simple_agent.run("What are its key features?", thread=thread_python)
print("Thread 1 continued:")
print(f"User: What are its key features?")
print(f"Agent: {response.text[:100]}...")
print("(Agent talks about Python, not JavaScript!)\n")

# Continue thread 2 - no Python context leakage
response = await simple_agent.run("What frameworks are popular?", thread=thread_javascript)
print("Thread 2 continued:")
print(f"User: What frameworks are popular?")
print(f"Agent: {response.text[:100]}...")
print("(Agent talks about JavaScript frameworks!)\n")

### Understanding Thread State

Threads in Agent Framework maintain conversation context automatically. You don't need to manually manage message history - the framework handles it for you:

In [ ]:
# Example 3: Understanding thread state
print("=== Example 3: Thread State ===\n")

# Threads maintain conversation context internally
# The Agent Framework handles message history automatically
# You don't need to manually access or manage messages

# Each call to agent.run() with a thread:
# 1. Adds your message to the thread's internal history
# 2. Sends the full conversation context to the model
# 3. Receives and stores the assistant's response
# 4. Returns the response to you

print("The thread object maintains internal state that enables:")
print("  ✓ Context retention across multiple turns")
print("  ✓ Natural conversation flow")
print("  ✓ No manual message management needed")
print("\nJust create a thread once and reuse it for the entire conversation!")

### Exercise: Build a Multi-Turn Conversation

**Your Task**: Create a conversation where:
1. Create a new thread
2. Ask the agent about a topic (e.g., "What is machine learning?")
3. Ask a follow-up question that relies on context (e.g., "What are its main types?")
4. Ask another follow-up (e.g., "Give me an example of supervised learning")
5. Observe how the agent maintains context

**Success Criteria**:
- The agent should understand context in follow-up questions without you repeating information
- Follow-up questions like "What are its types?" should be understood in context

In [ ]:
# Exercise: Your code here
# TODO: Create a multi-turn conversation following the instructions above

# Uncomment and complete:
# my_thread = simple_agent.get_new_thread()
# response = await simple_agent.run("Your first question", thread=my_thread)
# print(f"Agent: {response.text}")
# ... continue with follow-up questions ...

pass

<details>
<summary>💡 Click to see solution</summary>

```python
# Solution: Multi-turn conversation example
my_thread = simple_agent.get_new_thread()

# First question
response = await simple_agent.run("What is machine learning?", thread=my_thread)
print(f"Q1: What is machine learning?")
print(f"A1: {response.text}\n")

# Follow-up relying on context (notice we don't say "machine learning" again)
response = await simple_agent.run("What are its main types?", thread=my_thread)
print(f"Q2: What are its main types?")
print(f"A2: {response.text}\n")

# Another follow-up (still relying on context)
response = await simple_agent.run("Give me an example of supervised learning", thread=my_thread)
print(f"Q3: Give me an example of supervised learning")
print(f"A3: {response.text}\n")

print(f"✅ Conversation complete! The agent understood all context.")
```
</details>

## Summary

Microsoft Agent Framework provides:

1. **Simplicity**: 
   - Clean API with minimal setup
   - No complex configuration needed
   - Direct agent creation and usage

2. **Conversation Management**:
   - Built-in thread support
   - Stateful conversations out of the box
   - Easy multi-turn interactions

3. **Provider Flexibility**:
   - Switch providers by changing chat client
   - Same API works everywhere
   - No vendor lock-in

4. **Modern Design**:
   - Async-first architecture
   - Supports latest AI capabilities
   - Streaming support
   - Tool integration ready

## Next Steps

- Explore tool/function calling in the next notebook
- Learn about multi-agent orchestration
- Build production-ready applications with Agent Framework